In [ ]:
#pART 1: REGRESSION INTRODUCTION
!pip install quandl

In [1]:
import sklearn
#import quandl
import pandas as pd

#df= quandl.get('WIKI/GOOGL')

#print(df.head())

In [7]:
'''from google.colab import files

df.to_csv('df.csv')
files.download('df.csv')'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
!pip install PyDrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
df_path='/content/drive/My Drive/Datasets/Google stock prices/df.csv' 
df=pd.read_csv(df_path)
print(df.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend',
       'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close',
       'Adj. Volume'],
      dtype='object')


In [4]:
#Select columns that are needed
df= df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close','Adj. Volume']]

#Add a column with the values of high low percentage
df['HL_PCT']= (df['Adj. High']- df['Adj. Close'])/ df['Adj. Close'] * 100.0

#Calculate percentage change
df['PCT_change']= (df['Adj. Close']- df['Adj. Open'])/ df['Adj. Open'] * 100.0

#Re-define necessary columns
df= df[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']]

#df.head()



,Adj. Close,HL_PCT,PCT_change,Adj. Volume
0,50.322842,3.712563,0.324968,44659000.0
1,54.322689,0.710922,7.227007,22834300.0
2,54.869377,3.729433,-1.227880,18256100.0
3,52.597363,6.417469,-5.726357,15247300.0
4,53.164113,1.886792,1.183658,9188600.0


In [5]:
#PART 2: FEATURES AND LABELS
forecast_col= 'Adj. Close' #The name of the column that we will be forecasting  
df.fillna(-99999, inplace=True) #Fill all the places with missing values with -99999. inplace=True is used to modify the values in the origial dataframe and not return a copy 

import math
forecast_out= int(math.ceil(0.01*len(df))) #Predict value 1% of total days forward

df['label']= df[forecast_col].shift(-forecast_out)  #shifting rows of the column to be forecasted upwards

df.dropna(inplace=True)

print(df.head())
print(df.tail())

   Adj. Close    HL_PCT  PCT_change  Adj. Volume      label
0   50.322842  3.712563    0.324968   44659000.0  69.078238
1   54.322689  0.710922    7.227007   22834300.0  67.839414
2   54.869377  3.729433   -1.227880   18256100.0  68.912727
3   52.597363  6.417469   -5.726357   15247300.0  70.668146
4   53.164113  1.886792    1.183658    9188600.0  71.219849
      Adj. Close    HL_PCT  PCT_change  Adj. Volume    label
3384     1177.37  0.896914   -0.029718    1792602.0  1094.00
3385     1182.22  0.346805   -0.134312    1643877.0  1053.15
3386     1181.59  0.495942    0.476195    2774967.0  1026.55
3387     1119.20  1.081129   -0.729098    5798880.0  1054.09
3388     1068.76  4.325574   -2.893850    3742469.0  1006.94


In [ ]:
#PART 3: REGRESSION TRAINING AND TESTING
